# Autoencoders with feature extractors

In [ ]:
!pip install opencv-python datasets[vision] keras

In [ ]:
# import modules
import datasets
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm

import tensorflow as tf
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import (load_img, img_to_array)
from tensorflow.keras.applications.vgg16 import (VGG16, preprocess_input)

In [ ]:
def extract_features(img, model):
    """
    Extract features from image data using pretrained model (e.g. VGG16)

    Arguments:
    - img_path: path to image to extract features on
    - model: model to use as a feature extractor

    Returns:
    - A list of image embeddings for the input image

    Source:
        This function is taken from the Session 10 Notebook for the Visual Analytics Course.
    """
    # Define input image shape - remember we need to reshape
    #input_shape = (224, 224, 3)
    # load image from file path
    #img = load_img(img, target_size=(input_shape[0],
                                          #input_shape[1]))
    # convert to array
    img_array = img_to_array(img)
    # expand to fit dimensions
    expanded_img_array = np.expand_dims(img_array, axis=0)
    # preprocess image - see last week's notebook
    preprocessed_img = preprocess_input(expanded_img_array)
    # use the predict function to create feature representation
    features = model.predict(preprocessed_img)
    # flatten
    flattened_features = features.flatten()
    # normalise features
    normalized_features = flattened_features / norm(features)

    return normalized_features

In [ ]:
  model = VGG16(weights='imagenet', 
              include_top=False,
              pooling='avg',
              input_shape=(224, 224, 3))

In [ ]:
ds_resized = ds.map(_resize_img)

In [ ]:
feature_list = []

for i in range(len(ds_resized)):
    im = ds_resized[i]['image']
    feature = extract_features(im, model)
    feature_list.append(feature)

In [ ]:
features = np.asarray(feature_list)
features.shape

In [ ]:
def build_pretrained_autoencoder(features_shape, code_size, img_shape):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(features_shape))
    encoder.add(Flatten()) # flatten the tensor to 1D
    encoder.add(Dense(code_size)) # size of the final encoded vector (dense just means fully connected)

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size,)))
    decoder.add(Dense(np.prod(features_shape))) # np.prod(img_shape) is the same as 32*32*3, it's more generic than saying 3072 (returns the product)
    decoder.add(Reshape(img_shape))

    return encoder, decoder

def pretrained_autoencoder():
    #n_layers = 4
    input_img = tf.keras.layers.Input(shape=(512,), name='input')

    enc1 = tf.keras.layers.Dense(200, activation = 'relu', name='encoder1')(input_img)
    enc2 = tf.keras.layers.Dense(100, activation = 'relu', name='encoder2')(enc1)
    
    embedding = tf.keras.layers.Dense(30, activation = 'relu', name='encoder3')(enc2)
    
    dec1 = tf.keras.layers.Dense(100, activation = 'relu', name='decoder1')(embedding)
    dec2 = tf.keras.layers.Dense(200, activation = 'relu', name='decoder2')(dec1)

    decoded = tf.keras.layers.Dense(512, activation = 'relu', name='decoder3')(dec2)

    return tf.keras.models.Model(inputs=input_img, outputs=decoded, name='AE'), tf.keras.models.Model(inputs=input_img, outputs=embedding, name='encoder')



In [ ]:
autoencoder, encoder = pretrained_autoencoder()
autoencoder.summary()


In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(features, features, batch_size=256, epochs=200)

In [ ]:
feat_shape = features.shape[1]
im_shape = preprocessed_ds[0]['image'].shape
encoder, decoder = build_pretrained_autoencoder(feat_shape, 200, im_shape)

inp = Input(feat_shape)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = Model(inp, reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse')

print(autoencoder.summary())

In [ ]:
# vi definerer vores inputs undervejs, fordi vi ikke har sagt at det er en sequential model (hvor input ellers automatisk tages fra laget over)

def autoencoder(dims, act='relu', init='glorot_uniform'):
    n_stacks = len(dims) - 1
    # input
    input_img = tf.keras.layers.Input(shape=(dims[0],), name='input') # shape er en 1D vector som er længden af output af densenet
    x = input_img
    # internal layers of the encoder
    for i in range(n_stacks-1): # specificerer antallet af lag baseret på n_stacks variablen
        # så her, 3 lag
        # tror koden laver de tre lag (går dybere). det sidste (x) betyder, at det forrige x tages som input
        x = tf.keras.layers.Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # bottleneck
    # koden tager den sidste værdi i dims-vectoren (10) som længden på vektoren --> tager x som input
    encoded = tf.keras.layers.Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded # definerer ny input variabel
    # hidden layers of the decoder
    for i in range(n_stacks-1, 0, -1):
        x = tf.keras.layers.Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = tf.keras.layers.Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return tf.keras.models.Model(inputs=input_img, outputs=decoded, name='AE'), tf.keras.models.Model(inputs=input_img, outputs=encoded, name='encoder')

dims = [1024, 500, 500, 2000, 10] # forstår ikke hvad formålet med den her er 